In [1]:
!pip install annotated-images

  Preparing metadata (setup.py) ... done
  Created wheel for annotated-images: filename=annotated_images-0.1.4-py3-none-any.whl size=4277 sha256=c61f1cec9f3f678be41305bc99d7154346800649825a974207a46b84deef9622
  Stored in directory: /root/.cache/pip/wheels/f2/e3/4e/bb29b2b240a1b122d4eec3fc2c1c4e5b55ef499b7b201c6c09
Successfully built annotated-images


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import annotated_images

# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
annotated_images.split("/content/drive/MyDrive/BumpsandPotholes/models/dataset/", "/content/drive/MyDrive/BumpsandPotholes/models/", seed=1337, ratio=(.8, .2))

{'train': 374, 'test': 94, 'total': 468}

In [4]:
train_path = "/content/drive/MyDrive/BumpsandPotholes/models/train"

In [5]:
test_path = "/content/drive/MyDrive/BumpsandPotholes/models/test"

In [6]:
pip install labelme

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 46.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.8/270.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.7 MB/s eta 0:00:00
  Created wheel for labelme: filename=labelme-5.6.0-py3-none-any.whl size=1439635 sha256=db56a2d953

In [7]:
import os
import argparse
import json

from labelme import utils
import numpy as np
import glob
import PIL.Image


class labelme2coco(object):
    def __init__(self, labelme_json=[], save_json_path="./coco.json"):
        """
        :param labelme_json: the list of all labelme json file paths
        :param save_json_path: the path to save new json
        """
        self.labelme_json = labelme_json
        self.save_json_path = save_json_path
        self.images = []
        self.categories = []
        self.annotations = []
        self.label = []
        self.annID = 1
        self.height = 0
        self.width = 0

        self.save_json()

    def data_transfer(self):
        for num, json_file in enumerate(self.labelme_json):
            with open(json_file, "r") as fp:
                data = json.load(fp)
                if "imageData" in data:
                    self.images.append(self.image(data, num))
                    for shapes in data["shapes"]:
                        label = shapes["label"].split("_")
                        if label not in self.label:
                            self.label.append(label)
                        #points = shapes["points"]
                        points = [list(map(float, suba)) for suba in shapes["points"]]
                        self.annotations.append(self.annotation(points, label, num))
                        self.annID += 1

        # Sort all text labels so they are in the same order across data splits.
        self.label.sort()
        for label in self.label:
            self.categories.append(self.category(label))
        for annotation in self.annotations:
            annotation["category_id"] = self.getcatid(annotation["category_id"])

    def image(self, data, num):
        image = {}
        img = utils.img_b64_to_arr(data["imageData"])
        height, width = img.shape[:2]
        img = None
        image["height"] = height
        image["width"] = width
        image["id"] = num
        image["file_name"] = data["imagePath"].split("\\")[-1]  #windows
        print(image["file_name"])

        self.height = height
        self.width = width

        return image

    def category(self, label):
        category = {}
        category["supercategory"] = label[0]
        category["id"] = len(self.categories)
        category["name"] = label[0]
        return category

    def annotation(self, points, label, num):
        annotation = {}
        contour = np.array(points)
        x = contour[:, 0]
        y = contour[:, 1]
        area = 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))
        annotation["segmentation"] = [list(np.asarray(points).flatten())] #[list(map(float, np.asarray(points).flatten()))] #
        annotation["iscrowd"] = 0
        annotation["area"] = area
        annotation["image_id"] = num

        annotation["bbox"] = list(map(float, self.getbbox(points)))

        annotation["category_id"] = label[0]  # self.getcatid(label)
        annotation["id"] = self.annID
        return annotation

    def getcatid(self, label):
        for category in self.categories:
            if label == category["name"]:
                return category["id"]
        print("label: {} not in categories: {}.".format(label, self.categories))
        exit()
        return -1

    def getbbox(self, points):
        polygons = points
        mask = self.polygons_to_mask([self.height, self.width], polygons)
        return self.mask2box(mask)

    def mask2box(self, mask):

        index = np.argwhere(mask == 1)
        rows = index[:, 0]
        clos = index[:, 1]

        left_top_r = np.min(rows)  # y
        left_top_c = np.min(clos)  # x

        right_bottom_r = np.max(rows)
        right_bottom_c = np.max(clos)

        return [
            left_top_c,
            left_top_r,
            right_bottom_c - left_top_c,
            right_bottom_r - left_top_r,
        ]

    def polygons_to_mask(self, img_shape, polygons):
        mask = np.zeros(img_shape, dtype=np.uint8)
        mask = PIL.Image.fromarray(mask)
        xy = list(map(tuple, polygons))
        PIL.ImageDraw.Draw(mask).polygon(xy=xy, outline=1, fill=1)
        mask = np.array(mask, dtype=bool)
        return mask

    def data2coco(self):
        data_coco = {}
        data_coco["images"] = self.images
        data_coco["categories"] = self.categories
        data_coco["annotations"] = self.annotations
        return data_coco

    def save_json(self):
        print("save coco json")
        self.data_transfer()
        self.data_coco = self.data2coco()

        print(self.save_json_path)
        os.makedirs(
            os.path.dirname(os.path.abspath(self.save_json_path)), exist_ok=True
        )
        json.dump(self.data_coco, open(self.save_json_path, "w"), indent=4)

# Train JSON

In [8]:
trainjsons = glob.glob(os.path.join(train_path, "*.json"))

In [9]:
train_path

'/content/drive/MyDrive/BumpsandPotholes/models/train'

In [10]:
train_path1='/content/drive/MyDrive/BumpsandPotholes/models/'

In [11]:
labelme2coco(trainjsons, train_path1+str("/train.json"))

save coco json
RoadBumpImage130.jpg
RoadBumpImage63.jpg
PotholeImage54.jpg
RoadBumpImage200.jpg
PotholeImage155.jpg
RoadBumpImage109.jpg
PotholeImage129.jpg
PotholeImage47.jpg
PotholeImage24.jpg
RoadBumpImage223.jpg
RoadBumpImage227.jpg
RoadBumpImage187.jpg
PotholeImage85.jpg
RoadBumpImage24.jpg
RoadBumpImage178.jpg
RoadBumpImage14.jpg
PotholeImage152.jpg
RoadBumpImage184.jpg
PotholeImage74.jpg
RoadBumpImage163.jpg
PotholeImage43.jpg
RoadBumpImage230.jpg
PotholeImage128.jpg
RoadBumpImage82.jpg
PotholeImage165.jpg
RoadBumpImage268.jpg
PotholeImage181.jpg
PotholeImage61.jpg
PotholeImage70.jpg
PotholeImage179.jpg
RoadBumpImage158.jpg
RoadBumpImage73.jpg
PotholeImage142.jpg
PotholeImage75.jpg
RoadBumpImage13.jpg
PotholeImage33.jpg
RoadBumpImage153.jpg
RoadBumpImage56.jpg
RoadBumpImage124.jpg
RoadBumpImage236.jpg
PotholeImage56.jpg
RoadBumpImage228.jpg
PotholeImage95.jpg
PotholeImage157.jpg
RoadBumpImage71.jpg
PotholeImage73.jpg
PotholeImage159.jpg
RoadBumpImage147.jpg
RoadBumpImage26.jpg
P

In [16]:
trainjsons

['/content/drive/MyDrive/BumpsandPotholes/models/train/RoadBumpImage130.json',
 '/content/drive/MyDrive/BumpsandPotholes/models/train/RoadBumpImage63.json',
 '/content/drive/MyDrive/BumpsandPotholes/models/train/PotholeImage54.json',
 '/content/drive/MyDrive/BumpsandPotholes/models/train/RoadBumpImage200.json',
 '/content/drive/MyDrive/BumpsandPotholes/models/train/PotholeImage155.json',
 '/content/drive/MyDrive/BumpsandPotholes/models/train/RoadBumpImage109.json',
 '/content/drive/MyDrive/BumpsandPotholes/models/train/PotholeImage129.json',
 '/content/drive/MyDrive/BumpsandPotholes/models/train/PotholeImage47.json',
 '/content/drive/MyDrive/BumpsandPotholes/models/train/PotholeImage24.json',
 '/content/drive/MyDrive/BumpsandPotholes/models/train/RoadBumpImage223.json',
 '/content/drive/MyDrive/BumpsandPotholes/models/train/RoadBumpImage227.json',
 '/content/drive/MyDrive/BumpsandPotholes/models/train/RoadBumpImage187.json',
 '/content/drive/MyDrive/BumpsandPotholes/models/train/Pothol

# Test JSON

In [12]:
testjsons = glob.glob(os.path.join(test_path, "*.json"))

In [13]:
test_path

'/content/drive/MyDrive/BumpsandPotholes/models/test'

In [14]:
test_path1='/content/drive/MyDrive/BumpsandPotholes/models/'

In [15]:
labelme2coco(testjsons, test_path1+str("/test.json"))

save coco json
RoadBumpImage237.jpg
RoadBumpImage274.jpg
RoadBumpImage196.jpg
PotholeImage120.jpg
RoadBumpImage193.jpg
PotholeImage44.jpg
RoadBumpImage95.jpg
RoadBumpImage139.jpg
RoadBumpImage9.jpg
RoadBumpImage220.jpg
PotholeImage104.jpg
RoadBumpImage53.jpg
RoadBumpImage201.jpg
RoadBumpImage285.jpg
RoadBumpImage148.jpg
RoadBumpImage293.jpg
PotholeImage7.jpg
PotholeImage15.jpg
RoadBumpImage294.jpg
PotholeImage28.jpg
RoadBumpImage75.jpg
PotholeImage106.jpg
RoadBumpImage250.jpg
PotholeImage124.jpg
RoadBumpImage225.jpg
RoadBumpImage240.jpg
RoadBumpImage127.jpg
PotholeImage133.jpg
PotholeImage119.jpg
RoadBumpImage91.jpg
RoadBumpImage205.jpg
RoadBumpImage99.jpg
RoadBumpImage182.jpg
RoadBumpImage233.jpg
PotholeImage131.jpg
RoadBumpImage192.jpg
PotholeImage30.jpg
RoadBumpImage47.jpg
PotholeImage60.jpg
RoadBumpImage217.jpg
RoadBumpImage150.jpg
RoadBumpImage218.jpg
RoadBumpImage258.jpg
RoadBumpImage252.jpg
PotholeImage83.jpg
RoadBumpImage43.jpg
PotholeImage180.jpg
PotholeImage52.jpg
RoadBumpIma

In [17]:
testjsons

['/content/drive/MyDrive/BumpsandPotholes/models/test/RoadBumpImage237.json',
 '/content/drive/MyDrive/BumpsandPotholes/models/test/RoadBumpImage274.json',
 '/content/drive/MyDrive/BumpsandPotholes/models/test/RoadBumpImage196.json',
 '/content/drive/MyDrive/BumpsandPotholes/models/test/PotholeImage120.json',
 '/content/drive/MyDrive/BumpsandPotholes/models/test/RoadBumpImage193.json',
 '/content/drive/MyDrive/BumpsandPotholes/models/test/PotholeImage44.json',
 '/content/drive/MyDrive/BumpsandPotholes/models/test/RoadBumpImage95.json',
 '/content/drive/MyDrive/BumpsandPotholes/models/test/RoadBumpImage139.json',
 '/content/drive/MyDrive/BumpsandPotholes/models/test/RoadBumpImage9.json',
 '/content/drive/MyDrive/BumpsandPotholes/models/test/RoadBumpImage220.json',
 '/content/drive/MyDrive/BumpsandPotholes/models/test/PotholeImage104.json',
 '/content/drive/MyDrive/BumpsandPotholes/models/test/RoadBumpImage53.json',
 '/content/drive/MyDrive/BumpsandPotholes/models/test/RoadBumpImage201.j

...the end of split!